### GUI Interface

In [6]:
import tkinter as tk
from tkinter import simpledialog, messagebox

class MinimaxGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Minimax Tree Visualization")

        self.label = tk.Label(root, text="Enter the depth of the Minimax tree:")
        self.label.pack()

        self.depth_entry = tk.Entry(root)
        self.depth_entry.pack()

        self.submit_button = tk.Button(root, text="Create Tree", command=self.create_tree)
        self.submit_button.pack()

        self.tree_selection = tk.StringVar()
        self.tree_selection.set("Minimax")

        self.minimax_radio = tk.Radiobutton(root, text="Minimax Tree", variable=self.tree_selection, value="Minimax", command=self.update_canvas)
        self.minimax_radio.pack(anchor=tk.W)

        self.alpha_beta_radio = tk.Radiobutton(root, text="Alpha-Beta Pruning Tree", variable=self.tree_selection, value="AlphaBeta", command=self.update_canvas)
        self.alpha_beta_radio.pack(anchor=tk.W)

        self.canvas = tk.Canvas(root, width=1000, height=700, bg='white')
        self.canvas.pack()

    def create_tree(self):
        try:
            depth = int(self.depth_entry.get())
            if depth <= 0:
                raise ValueError("Depth must be positive")
            self.initialize_trees(depth)
        except ValueError as e:
            messagebox.showerror("Input Error", str(e))

    def initialize_trees(self, depth):
        self.canvas.delete("all")
        total_leaf_nodes = 2**depth
        leaf_values_str = simpledialog.askstring("Input", f"Enter {total_leaf_nodes} leaf node values separated by spaces:")
        leaf_values = list(map(int, leaf_values_str.split()))
        
        if len(leaf_values) != total_leaf_nodes:
            messagebox.showerror("Input Error", f"Exactly {total_leaf_nodes} values are required.")
            return

        self.minimax_root = build_tree(leaf_values.copy(), depth)
        self.alpha_beta_root = build_tree(leaf_values.copy(), depth)

        minimax(self.minimax_root)
        alpha_beta_pruning(self.alpha_beta_root)

        self.update_canvas()

    def update_canvas(self):
        self.canvas.delete("all")
        if self.tree_selection.get() == "Minimax":
            self.draw_tree(self.minimax_root, 500, 50, 200)
        else:
            self.draw_tree(self.alpha_beta_root, 500, 50, 200)

    def draw_tree(self, node, x, y, offset):
        if node is None:
            return
        
        node_type = "Max" if node.is_max else "Min"
        if node.is_max:
            self.canvas.create_oval(x - 15, y - 15, x + 15, y + 15, fill='lightblue')
        else:
            self.canvas.create_rectangle(x - 15, y - 15, x + 15, y + 15, fill='lightgreen')
        
        self.canvas.create_text(x, y, text=str(node.value), fill="black")
        self.canvas.create_text(x, y - 20, text=node_type, fill="black")

        if node.left:
            if node.left.pruned:
                self.canvas.create_line(x, y, x - offset, y + 50, fill='red', dash=(2, 2))
            else:
                self.canvas.create_line(x, y, x - offset, y + 50)
            self.draw_tree(node.left, x - offset, y + 50, offset // 2)
        
        if node.right:
            if node.right.pruned:
                self.canvas.create_line(x, y, x + offset, y + 50, fill='red', dash=(2, 2))
            else:
                self.canvas.create_line(x, y, x + offset, y + 50)
            self.draw_tree(node.right, x + offset, y + 50, offset // 2)

if __name__ == "__main__":
    root = tk.Tk()
    app = MinimaxGUI(root)
    root.mainloop()


### Minimax Tree Creation

In [5]:
class Node:
    def __init__(self, value=None, is_max=True):
        self.value = value
        self.is_max = is_max
        self.left = None
        self.right = None
        self.pruned = False

def build_tree(values, depth, is_max=True):
    if depth == 0:
        return Node(value=values.pop(0), is_max=is_max)
    
    node = Node(is_max=is_max)
    node.left = build_tree(values, depth - 1, not is_max)
    node.right = build_tree(values, depth - 1, not is_max)
    
    return node

### Minimax Computation

In [4]:
def minimax(node, is_max=True):
    if node.left is None and node.right is None:
        return node.value
    
    if is_max:
        max_eval = -float('inf')
        max_eval = max(max_eval, minimax(node.left, False))
        max_eval = max(max_eval, minimax(node.right, False))
        node.value = max_eval
        return max_eval
    else:
        min_eval = float('inf')
        min_eval = min(min_eval, minimax(node.left, True))
        min_eval = min(min_eval, minimax(node.right, True))
        node.value = min_eval
        return min_eval

### Alpha-Beta Pruning

In [3]:
def alpha_beta_pruning(node, alpha=-float('inf'), beta=float('inf'), is_max=True):
    if node.left is None and node.right is None:
        return node.value
    
    if is_max:
        max_eval = -float('inf')
        max_eval = max(max_eval, alpha_beta_pruning(node.left, alpha, beta, False))
        alpha = max(alpha, max_eval)
        if alpha >= beta:
            node.value = max_eval
            node.right.pruned = True
            return max_eval
        max_eval = max(max_eval, alpha_beta_pruning(node.right, alpha, beta, False))
        node.value = max_eval
        return max_eval
    else:
        min_eval = float('inf')
        min_eval = min(min_eval, alpha_beta_pruning(node.left, alpha, beta, True))
        beta = min(beta, min_eval)
        if alpha >= beta:
            node.value = min_eval
            node.right.pruned = True
            return min_eval
        min_eval = min(min_eval, alpha_beta_pruning(node.right, alpha, beta, True))
        node.value = min_eval
        return min_eval

# Minimax Tree Visualization with Alpha-Beta Pruning

### Objective

### Assignment Components

### Implementation Details 

### Challenges Faced 

### Additional Features